In [1]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
import csv
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

In [2]:
def extractQuestionTitle(driver):
    try:
        question_title = driver.find_element_by_xpath('.//span[contains(@class, "ui_qtext_rendered_qtext")]').text
    except Exception as e:
        question_title = "error"
    return question_title

In [3]:
def extractTopics(driver): # modified
    topic_list = []
    
    try:
        topic_area = driver.find_element_by_xpath(".//div[contains(@class,'QuestionTopicHorizontalList')]")
    except:
        topic_list = ['error']
        return topic_list
    
    topic_list = topic_area.text.split('\n')
           
    return topic_list

In [4]:
def scrollDownPage(driver, speed):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

In [5]:
def extractRespondContent(driver):
    try:
        respond_content = driver.find_element_by_xpath(".//div[@class = 'q-text']").text
    except Exception as e:
        respond_content = "error"
        
    return respond_content

In [6]:
def extractRespondViewAndUpvote(driver):
    try:
        respond_stat = driver.find_element_by_xpath(".//div[@class = 'q-text qu-mt--small qu-color--gray_light qu-fontSize--small qu-passColorToLinks']").text
        
        view_text = respond_stat.split(" view")[0]
        if "k" in view_text:
            respond_view = int(float((view_text.split("k"))[0]) * 1000)
        elif "m" in view_text:
            respond_view = int(float((view_text.split("m"))[0]) * 1000000)
        else:
            respond_view = int(view_text)
        
        respond_upvote = 0
        if "Upvoter" in respond_stat:
            upvote_text = (respond_stat.split("View ")[1]).split(" ")[0]
            if "k" in upvote_text:
                respond_upvote = int(float((upvote_text.split("k"))[0]) * 1000)
            elif "m" in upvote_text:
                respond_upvote = int(float((upvote_text.split("m"))[0]) * 1000000)
            else:
                respond_upvote = int(upvote_text)
        else:
            respond_upvote = 0
            
    except:
        respond_view = 0
        respond_upvote = 0
        
    return respond_view, respond_upvote

In [7]:
def extractDate(driver):
    try:
        log = driver.find_element_by_xpath(".//p[@class = 'log_action_bar']")
        date = log.text.split("· ")[1]
    except:
        date = "error"
    return date

In [8]:
def extractAuthor(driver):
    block_header = driver.find_element_by_xpath (".//div[@class = 'feed_item_activity']")
    try:
        user_name = block_header.find_element_by_xpath(".//span[@class = 'anon_user']").text
    except Exception:
        try:
            user_name = block_header.find_element_by_xpath(".//a[@class = 'user']").text
        except Exception as e:
            user_name = "error"
    
    return user_name

In [9]:
def log_stat_extraction(driver, driver2):
    # return element
    respond_author_list = []
    respond_content_list = []
    respond_upvote_list = []
    respond_view_list = []
    respond_date_list = []
    respond_count = 0
    question_author = ''
    question_create_time = ''
    
    try:
        block_list = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")
    except:
        respond_author_list =  ["error"]
        respond_content_list = ["error"]
        respond_upvote_list = ["error"]
        respond_view_list = ["error"]
        respond_date_list = ["error"]
        respond_count = 0
        question_auther = "error"
        question_create_time = "error"
        
        return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_auther, question_create_time
        
    for block in block_list:
        block_error = []
        
        block_header = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']").text
        
        try:
            block_revision = block.find_element_by_xpath (".//div[@class = 'revision']").text
        except:
            continue
            
        if 'answer has since been deleted' in block_revision:
            continue
        elif 'deleted' in block_header:
            continue
        
        if 'Answer' in block_header:
            # click the answer link
            try:
                url2 = block.find_element_by_xpath (".//div[@class = 'feed_item_activity']/a").get_attribute('href')
                driver2.get(str(url2))
                respond_count += 1
            except Exception as e:
                continue
                
            # extract author name(log page and driver 1)
            respond_author = extractAuthor(block)
            respond_author_list.append(respond_author)
            
            # extract content
            respond_content = extractRespondContent(driver2)
            respond_content_list.append(respond_content)
                
            # extract views and upvote
            respond_view, respond_upvote = extractRespondViewAndUpvote(driver2)
            respond_view_list.append(respond_view)
            respond_upvote_list.append(respond_upvote)
                
            # extract date(log page and driver 1)
            respond_date = extractDate(block)
            respond_date_list.append(respond_date)
            
        elif 'Question added by' in block_header:
            # extract author name
            question_author = extractAuthor(block)
            
            # extract date
            question_create_time = extractDate(block)

    return respond_author_list, respond_content_list, respond_upvote_list, respond_view_list, respond_date_list, respond_count, question_author, question_create_time



In [10]:
def extractNoOfFollowerAndView(driver):
    try:
        stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
        no_of_follower = int((stats[0].text.split(' '))[0])
        no_of_view = (stats[1].text.split(' '))[0]
    except:
        no_of_follower = 0
        no_of_view = 0
        
    return no_of_follower, no_of_view

In [11]:
def extractQuestion(driver, driver2, url):
    data = {}
    print(url)
    driver.get(url+'/log')
    data['question_title'] = extractQuestionTitle(driver)
    data['topic_list'] = extractTopics(driver)
    
    scrollDownPage(driver, speed = 5)
    data['name_list'], data['ans_list'], data['upvote_list'], data['view_list'], data['date_list'], data['respond_collected'], data['question_author'], data['create_time'] = log_stat_extraction(driver, driver2)
    
    data['no_of_follower'], data['no_of_view'] = extractNoOfFollowerAndView(driver)
    return data

In [12]:
def readTopics():
    with open('topics.txt') as topics:
        topic_array = topics.readlines()
    topic_array = [topic.strip("\n") for topic in topic_array] 
    print(topic_array)
    return topic_array    

In [13]:
def readURLs(file):
    with open(file) as URLs:
        URL_array = URLs.readlines()
    URL_array = [URL.strip("\n") for URL in URL_array]
    return URL_array

In [14]:
def connectChrome():
    # instantiate a chrome options object so you can set the size and headless preference
    options = Options()
    options.add_argument("--headless")
    options.add_argument("log-level=3")
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_script("window.scrollTo(0, 0);")
    # download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    # and put it in the current directory
    chromedriver = "/home/youcef/Documents/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver     
    browser = webdriver.Chrome(options=options) 
    browser.get("https://www.quora.com/")
    time.sleep(2)
    return browser

In [15]:
def crawlURLByTopic(topic):
    file_location = os.path.join(os.getcwd() + "/topics/" + topic + "_question_urls.txt")
    try:
        with open(file_location) as f:
            print("URLs of the topic \"" + topic + "\" has been crawled")
    except:
        print("Start crawling the URLs of the topic \"" + topic + "\"")
        URLCount = 0
        driver = connectChrome()
        QuoraURL = "https://www.quora.com/topic/" + topic + "/all_questions"
        print(QuoraURL)
        
        try:
            driver.get(QuoraURL)
        except Exception as e0:
            print("exception e0")
            print("Error on line {}".format(sys.exc_info()[-1].tb_lineno), type(e0).__name__, e0)
            return   
        
        # define pause time for browser
        SCROLL_PAUSE_TIME = 2
        
        # get browser source
        html_source = driver.page_source
        question_count_soup = BeautifulSoup(html_source, "html.parser")

        #  get total number of questions
        question_count_str = question_count_soup.find("a", attrs = {"class": "TopicQuestionsStatsRow"})
        if str(question_count_str) == "None":
            print("Topic does not exist in Quora, so we will skip it.")
            return
        question_count = question_count_str.contents[0].text 
        if "k" in str(question_count):
            question_count = str(question_count).replace("k", "")
            question_count = int(float(question_count) * 1000)
        print("Number of questions for this topic : "+ str(question_count))
        if str(question_count) == "0":            
            print("Topic has 0 question, so we will skip it.")
            return
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        question_set = set()
        # infinite while loop, break it when you reach the end of the page or not able to scroll further.
        if int(question_count) > 10:
            start_time_sd = time.time()
            max_time = int(question_count) * 0.25
            if int(question_count) > 8000:
                max_time = 1800
            while True:
                i = 0
                # try to scroll 5 times in case of slow connection
                while True:
                    # Scroll down to one page length
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    # Wait to load page
                    time.sleep(SCROLL_PAUSE_TIME)

                    # get page height in pixels
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    # break this loop when you are able to scroll futher
                    if new_height != last_height:
                        break
                    if i == 1:
                        SCROLL_PAUSE_TIME = 5   
                    if i == 2:
                        if int(question_count) > 500 and int(question_count) < 2000:
                            SCROLL_PAUSE_TIME = 8
                        elif int(question_count) > 2000:
                            SCROLL_PAUSE_TIME = 16    
                        elif int(question_count) < 500:
                            break
                    if i == 3:
                        break
                    i += 1
                SCROLL_PAUSE_TIME = 2    
                # not able to scroll further, break the infinite loop
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                total_time = time.time() - start_time_sd
                if total_time > max_time:
                    print("Max time exceeded")
                    break
        print('getting ressources ...')
        # get html page source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        # question_link is the class for questions
        question_link = soup.find_all("a", attrs = {"class": "question_link"}, href = True)

        # add questions to a set for uniqueness
        for ques in question_link:
            question_set.add(ques)
            
        # write content of set to a file called question_urls.txt
        with open(file_location, 'w') as topic_urls:
            writer = csv.writer(topic_urls)
            for ques in question_set:
                link_url = "http://www.quora.com" + ques.attrs['href']
                writer.writerows([[link_url]])
            

In [16]:
def crawlQuestionByTopic(topic):
    start_time = time.time()
    try:
        topic_url_file = (os.path.join(os.getcwd()+"/topics/" + topic + "_question_urls.txt"))
        print(topic_url_file)
        # Open question url file
        file_question_urls = open(topic_url_file, mode = 'r')
        answerFile = topic + "_answers.txt"

        #file_users = open(os.path.join(sys.path[0]+"users.txt", mode = 'a'))
        current_lines = file_question_urls.readlines()
        URLs = readURLs(topic_url_file)
        driver = connectChrome()
        driver2 = connectChrome()
        topic_array = {}
        topic_array["topic"] = topic
        topic_array["no_of_follows"] = 0
        questions = {}
        for URL in URLs:
            question_array = extractQuestion(driver, driver2, URL)
            questions[str(len(questions))] = question_array
            #print(question_array)
            topic_array["questions"] = questions
            crawledData = json.dumps(topic_array)
            with open(answerFile, "w") as file_answers:
                file_answers.write(crawledData)
                file_answers.close()
        return topic
    except:
        return

In [17]:
def main():
    topics = readTopics()
    for topic in topics:
        crawlURLByTopic(topic)
        crawlQuestionByTopic(topic)
    
if __name__ == "__main__": main()    

['Visiting-and-Travel', 'Business', 'Psychology', 'History', 'Cooking', 'Photography', 'Sports', 'Design', 'Fashion-and-Style', 'Writing', 'Economics', 'Mathematics', 'Philosophy', 'Politics', 'Finance', 'Marketing', 'Television-series', 'Fine-art', 'Literature', 'Entertainment', 'Journalism', 'Healthy-Eating']
Start crawling the URLs of the topic "Visiting-and-Travel"
https://www.quora.com/topic/Visiting-and-Travel/all_questions
Number of questions for this topic : 0
Topic has 0 question, so we will skip it.
/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping/topics/Visiting-and-Travel_question_urls.txt
Start crawling the URLs of the topic "Business"
https://www.quora.com/topic/Business/all_questions
Topic does not exist in Quora, so we will skip it.
/home/httpak/Downloads/Quora-and-Twitter-crawler-and-user-matcher-master/1-Quora-scrapping/topics/Business_question_urls.txt
Start crawling the URLs of the topic "Psychology"
https://www.quora.com/to

http://www.quora.com/unanswered/Which-one-is-better-in-terms-of-job-and-salary-stress-a-software-engineer-or-an-MSc-in-psychology
http://www.quora.com/Why-cant-a-person-take-a-decision-during-a-dangerous-time
http://www.quora.com/unanswered/What-is-the-difference-between-a-full-blown-versus-a-borderline-sociopath
http://www.quora.com/Why-do-people-watch-tragedies
http://www.quora.com/unanswered/What-could-lead-someone-to-be-1-Gluttonous-2-Lustful-By-gluttonous-I-am-referring-to-overindulgence-while-with-Lustful-I-am-referring-to-the-want-for-something-even-in-a-twisted-form
http://www.quora.com/What-is-the-difference-between-psychology-and-dark-psychology
http://www.quora.com/unanswered/Which-Mumbai-University-college-can-I-get-into-for-a-BA-in-psychology-with-80-in-HS
http://www.quora.com/If-a-cure-is-available-for-covid19-will-we-still-be-affected-by-social-distancing
http://www.quora.com/Why-are-dreams-so-sweet-but-reality-so-cruel
http://www.quora.com/How-do-I-reduce-DID-cases-from

http://www.quora.com/unanswered/What-is-the-experimental-method-in-educational-psychology
http://www.quora.com/Does-nutrition-have-an-effect-on-mood-and-can-it-be-immediate
http://www.quora.com/unanswered/How-does-sensory-motor-amnesia-theory-fit-with-what-is-accepted-in-psychology-and-neuroscience
http://www.quora.com/Are-conspiracy-theories-surrounding-COVID-19-a-comfort-seeking-behaviour
http://www.quora.com/How-can-somebody-force-himself-to-enjoy-programming-when-sitting-in-front-of-a-computer-screen-feels-like-being-suffocated
http://www.quora.com/Do-psychopaths-enjoy-being-around-others-that-are-also-psychopaths
http://www.quora.com/Which-are-the-best-universities-in-Ontario-to-study-for-an-MA-in-clinical-psychology
http://www.quora.com/unanswered/Has-covid-19-helped-broken-relationships-Has-it-affected-positively-during-this-difficult-times
http://www.quora.com/What-s-your-interpretation-of-Joe-Biden-s-cognitive-state-I-ve-seen-many-primary-sources-speeches-debates-and-interview

http://www.quora.com/As-someone-who-has-court-tomorrow-with-their-undiagnosed-bpd-ex-current-coparent-what-should-I-do-in-terms-of-bringing-forth-proofs-of-the-excruitiatingly-blatant-insulting-purjery-she-has-committed-on-her-reply-to
http://www.quora.com/What-are-symptoms-of-Aspergers-comorbid-with-covert-narcissistic-personality-disorder
http://www.quora.com/Why-do-actors-put-themselves-through-so-much-mental-and-physical-stress-for-certain-roles
http://www.quora.com/unanswered/Why-isn-t-there-a-lot-of-research-or-stories-on-intermittent-explosive-disorder
http://www.quora.com/unanswered/Is-there-any-mental-illness-that-makes-you-find-horrific-things-funny
http://www.quora.com/unanswered/What-is-the-scope-or-how-is-it-to-be-a-neuropsychologist-after-taking-physics-as-your-BS
http://www.quora.com/Is-it-normal-to-dislike-music-because-it-once-helped-you-feel-good
http://www.quora.com/What-are-some-books-which-explain-the-INFP-mentality
http://www.quora.com/How-difficult-or-easy-is-it-

http://www.quora.com/What-do-you-think-about-the-emerging-field-of-neuropsychoanalysis-or-the-intersection-of-neuroscience-and-psychoanalysis
http://www.quora.com/I-feel-used-up-as-if-I-m-tired-of-existing-but-there-was-no-recent-trauma-that-couldve-made-me-feel-that-way-Why-do-I-feel-this
http://www.quora.com/Why-do-our-core-beliefs-shape-our-emotions-and-what-about-our-emotions-is-energy-we-gather-to-shape-our-core-beliefs
http://www.quora.com/Did-William-James-create-the-first-psychology-lab-in-Germany
http://www.quora.com/unanswered/How-much-has-psychology-discovered-I-mean-like-theres-a-lot-of-development-in-other-sub-like-nanology
http://www.quora.com/unanswered/What-is-virtual-post-traumatic-stress-disorder-v-PTSD-What-causes-it
http://www.quora.com/Do-people-realize-that-by-not-wearing-a-mask-they-are-putting-other-people-in-danger
http://www.quora.com/unanswered/Why-after-an-intense-hard-workout-I-feel-no-fear-and-feel-fully-relaxed-Is-there-a-scientific-explanation-of-what-ac

...

http://www.quora.com/What-is-the-maximum-score-you-can-take-on-a-golf-hole
http://www.quora.com/unanswered/Why-is-the-China-Open-so-important-to-badminton
http://www.quora.com/Who-would-win-a-fight-between-Manny-Pacquiao-and-Joel-Embiid
http://www.quora.com/Why-is-lacrosse-in-America-often-described-as-an-elitist-sport
http://www.quora.com/Is-squash-an-elite-sport
http://www.quora.com/unanswered/How-do-I-start-working-on-my-vertical-jumps-being-a-beginner-What-are-things-I-have-to-keep-in-mind-to-not-waste-time-I-need-a-full-guide-I-am-16-y-o-THANKS
http://www.quora.com/unanswered/What-are-some-stunning-photos-of-women-playing-sports
http://www.quora.com/unanswered/Did-ex-ESPN-broadcaster-Marcellus-Wiley-really-compare-gangsta-rapper-Snoop-Doog-to-infamous-radio-host-Don-Imus-Why-would-he-do-that
http://www.quora.com/If-the-Chicago-Bulls-had-kept-the-team-together-for-one-more-season-after-winning-their-6-championships-in-the-97-98-season-do-you-think-they-could-have-won-the-championsh

http://www.quora.com/Who-is-number-25-on-the-Denver-Nuggets
http://www.quora.com/How-can-they-open-up-stadiums-for-sports-spectators-during-this-COVID-19-pandemic
http://www.quora.com/How-fast-is-Adama-Traore
http://www.quora.com/unanswered/How-important-was-Dennis-Rodman-to-the-Bulls-final-championship
http://www.quora.com/Ive-heard-Floyd-Mayweather-is-coming-out-of-retirement-yet-again-If-so-do-you-think-he-will-have-a-rematch-against-Saul-Canelo-Alvarez
http://www.quora.com/unanswered/How-do-I-become-a-volleyball-coach-in-India-Is-there-any-coaching-certificate
http://www.quora.com/How-long-has-tennis-been-in-the-Olympics
http://www.quora.com/Did-Trump-play-any-sports-when-he-was-at-school
http://www.quora.com/Which-is-the-best-nutrition-product-for-a-sports-person
http://www.quora.com/unanswered/Misha-Grewal-is-associated-with-which-games-sports
http://www.quora.com/Would-Rocky-Balboa-defeat-Tyson-Fury
http://www.quora.com/Who-was-the-referee-for-the-2015-Rugby-World-Cup-final
http

http://www.quora.com/Who-would-have-gotten-the-best-career-if-they-had-avoided-injuries-Grant-Hill-Penny-Hardaway-or-Bill-Walton
http://www.quora.com/unanswered/What-athletes-seem-to-play-better-when-hurt
http://www.quora.com/unanswered/What-was-the-first-softball-team
http://www.quora.com/Who-is-your-favorite-Grand-Prix-Dressage-horse-past-or-present-and-why
http://www.quora.comhttps://www.quora.com/q/aqpgfyteauzriomo/What-if-the-steelers-had-picked-dan-marino-ahead-of-the-dolphins-in-the-1983-NFL-draft
http://www.quora.com/unanswered/How-do-you-keep-your-weight-on-the-front-foot-of-golf
http://www.quora.com/What-are-some-of-the-best-sports-to-do-on-the-prairies-or-in-extremely-flat-and-potentially-windy-areas
http://www.quora.com/unanswered/Did-the-size-of-the-basketball-contribute-to-Croatia-in-the-1992-Olympic-game-against-the-US-sports-history
http://www.quora.com/unanswered/Where-do-you-find-the-number-of-consecutive-games-in-a-row-at-Batista-Ice-Hockey
http://www.quora.com/What-

http://www.quora.com/Who-has-the-best-plus-minus-rating-in-Stanley-Cup-playoff-history
http://www.quora.com/Why-do-athletes-have-low-blood-pressure
http://www.quora.com/Do-you-think-the-2020-Ryder-Cup-should-be-postponed-until-a-later-date-so-that-fans-can-be-in-attendance
http://www.quora.com/Who-has-won-the-most-womens-tennis-majors
http://www.quora.com/unanswered/Does-Rainn-Wilson-play-ping-pong-as-well-as-Dwight-in-The-Office
http://www.quora.com/unanswered/Which-of-the-following-is-not-an-Olympic-sport-basketball-cricket-athletics-or-swimming
http://www.quora.com/unanswered/Are-all-nitrile-gloves-the-same
http://www.quora.com/Who-won-the-most-consecutive-Wimbledon-singles-titles
http://www.quora.com/If-a-lot-of-the-US-is-made-up-of-European-and-Latin-American-immigrants-why-arent-international-proper-sports-popular
http://www.quora.com/How-many-rounds-are-in-a-double-elimination-tournament
http://www.quora.com/Can-you-break-any-of-the-world-records-In-which-category-does-that-fall

http://www.quora.com/If-your-state-opened-up-for-business-would-you-feel-comfortable-going-out-to-restaurants-malls-movies-and-sports-events
http://www.quora.com/unanswered/What-does-Fighting-is-an-athletic-skill-and-like-any-other-athlete-a-good-fighter-develops-instincts-and-ingrained-muscle-memory-mean
http://www.quora.com/unanswered/What-athlete-has-appeared-on-the-Wheaties-box-the-most
http://www.quora.com/What-is-a-10-point-tiebreaker-in-tennis
http://www.quora.com/unanswered/Why-is-there-no-professional-tetherball
http://www.quora.com/unanswered/Why-are-there-many-Asian-clubs-in-the-Russian-hockey-league-KHL-Континентальная-хоккейная
http://www.quora.com/unanswered/How-much-do-Sport-Clips-stylists-make
http://www.quora.com/How-many-Grand-Slam-finals-has-Roger-Federer-lost
http://www.quora.com/unanswered/What-should-I-do-for-my-nose-which-got-crooked-after-an-accident-in-sports
http://www.quora.com/unanswered/Who-famously-won-555-matches-consecutively-in-their-chosen-sport
http:/